In [1]:
#default_exp full_volume

In [2]:
#export
#dependencies

#nlp packages
import spacy
from spacy.util import minibatch, compounding

#manipulation of tables/arrays
import pandas as pd
import numpy as np
import copy
import json

import difflib

#internal imports
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.model_performance_utils import *
from ssda_nlp.xml_parser import *
from ssda_nlp.unstructured2markup import *
from ssda_nlp.utility import *
from ssda_nlp.relationships import *

In [3]:
#export

def check_unassigned(entities, isVerbose=0):
    char_df = entities[(entities['pred_label'] == 'CHAR')]
    char_df = copy.deepcopy(char_df[char_df['assigned'] == False])
    rel_df = entities[(entities['pred_label'] == 'REL')]
    rel_df = copy.deepcopy(rel_df[rel_df['assigned'] == False])

    unassigned_df = char_df.append(rel_df)
    unassigned_df.sort_values(by='index', inplace = True)
    unassigned_df.drop(['index'], axis=1, inplace = True)
    unassigned_df.reset_index(drop = True, inplace = True)
    
    if isVerbose:
        print("entities:")
        display(entities.head(1))
        print(entities.shape)
        print("------------------------------------------------")
        
        print("Unassigned CHAR/RELs found:")
        print("char_df shape: " + str(char_df.shape))
        print("rel_df shape: " + str(rel_df.shape))
        print("unassigned_df shape: " + str(unassigned_df.shape))
        print("------------------------------------------------")

        print("unassigned_df")
        if unassigned_df.shape[0] < 1:
            print("No unassigned characteristics or relationships found...")
        else:
            display(unassigned_df.head())

        print("noCategory")
        if noCategory.shape[0] < 1:
            print("No characteristics without categories found...")
        else:
            display(noCategory.head())
            
    return unassigned_df

def check_lengths(name_list, idx, idx2):
    name1 = name_list[idx]
    name2 = name_list[idx2]
    if len(name1)<=(len(name2)+1) and len(name1)>=(len(name2)-1):
        my_ratio = int(difflib.SequenceMatcher(None, name1, name2).ratio()*100)
        if my_ratio > 80:
            #Then these could potentially be the same name but with small differences
            return True
    return False

In [4]:
#no_test

#Original pseudo code for validation:

#if entry_type == "baptism":
        #is there a baptism event?
        #if so:
            #does baptism event have a complete date?
            #does baptism event have a location?
        
        #is there an identified cleric?
        
        #is there an identified principal?
        #if so:
            #are principal's godparent(s) identified?
        #if so, and if principal is an infant:                
            #are principal's parent(s) identified?
            #is there a birth event?
            #if so:
                #does birth event have a complete date?
                #does birth event have a location?
        #if so, and if principal is enslaved:
            #is principal's enslaver identified?
            
        #other questions re people:
        #are there any entities labeled PER who do not have an explicit role in the baptism (e.g. principal, cleric, parent, enslaver, etc.)
        #are there likely couples (e.g. parents, godparents) not explicitly flagged as such
        #are there any people with very similar names that still appear separately
        
        #questions re characteristics:
        #are there characteristics that were not categorized
        #are there characteristics or relationships that were not assigned
        #for ethnicities that were assigned, confirm that they were *not* assigned to slaveholders or clerics 
        
#My pseudocode was misleading, the cleric check is independent of principal sub-checks (I adjusted)
#Is it possible for principal/cleric to be assigned in the absence of a defined event? Is it possible for there not to be a defined event? Results here make it look like the answer to one or both of these is no (why? I need to check). If/when the answer to one or both is yes, validate_entry will need to be refactored (since it will break due to undefined vars).
#I think that the issue with hasEnslaver is that the previous approach to baptism_princ was typically grabbing the reference to the principal at the beginning of each entry, whereas a subsequent reference was typically the one that was linked to the enslaver. If so, this should be fixed by the above.
#              Spoiler: it didn’t
#General note: some of the “desirable” checks are going to be low b/c they are conditional to start with (i.e. we don’t care if there isn’t a birth event if the principal is an adult).
#I think there’s a problem with the hasWrongEthAssgnt_ensl check. As previously written, I believe that it flags if the person in question is an enslaver and that person’s slave has an ethnicity. I’ve flipped the second condition but left the original statement commented in case I’ve missed something b/c it doesn’t look like it was flagging as much as I would expect if it was incorrect in the way that I’m interpreting.


In [5]:
#export

def validate_entry(entry_entities, entry_people, entry_places, entry_events, uncategorized_characteristics, all_first_names, isVerbose = 0, entry_type = "baptism"):
    '''
    analyzes data extracted from a single ecclesiastical entry to assess accuracy of automated entity and relationship extraction
        entry_type: baptism, marriage, or burial
        entry_entities: df containing all entities extracted from entry (with assignment status)
        entry_people: list of dictionaries, each containing people who appear in entry
        entry_places: list of dictionaries, each containing places that appear in entry
        entry_events: list of dictionaries, each containing events that appear in entry
        
        returns: dictionary in which key-value pairs encode various details regarding performance of automation
    '''
    
    validation_dict = {}
    
    #Reinitialize all booleans
    #Original base checks related to the event
    isBaptism=0; isDateComplete=0; hasLocation=0; hasCleric=0; hasPrincipal=0; 
    hasRelations=0; hasGodparents=0; hasEnslaver=0;
    isInfant=0; hasParents=0; isBirthEvent=0; isDateComplete_birth=0; hasLocation_birth=0;
    isEnslaved=0; similarNames=0;
    #"Extra" checks related to people
    hasWrongEthAssgnt_ensl = 0; hasWrongEthAssgnt_cler = 0; hasUncatChars = 0; hasUnassgnEnts = 0
    hasUncoupledParents = 0; hasUncoupledGodparents = 0; hasUnrelatedPersons=0
    
    my_keys = ['isBaptism', 'isDateComplete', 'hasLocation', 'hasCleric', 'hasPrincipal',
              'hasEnslaver', 'isEnslaved', 'hasRelations', 'hasGodparents', 
              'isInfant', 'hasParents', 'isBirthEvent', 'isDateComplete_birth', 'hasLocation_birth', 
              'hasUnrelatedPersons', 'hasUncoupledParents', 'hasUncoupledGodparents','similarNames', 
              'hasWrongEthAssgnt_ensl', 'hasWrongEthAssgnt_cler', 'hasUncatChars', 'hasUnassgnEnts']
    
    if isVerbose:
        print("Entry entities:")
        display(entry_entities.head())
        print("Entry people:")
        display(entry_people)
        print("Entry places:")
        display(entry_places)
        print("Entry events:")
        display(entry_events)
        #print("Uncategorized characteristics:") #This is a dataframe
    
    if entry_type == "baptism":
        #is there a baptism event? ##############################################################################
        baptism_event = {} #Initialization
        for bapt_idx in range(len(entry_events)):
            isBaptism = entry_events[bapt_idx].get('type')=='baptism'
            if isBaptism:
                baptism_event = entry_events[bapt_idx]
                break
        
        #if so:
        if isBaptism:
            #does baptism event have a complete date? #########################################################################
            event_date = baptism_event.get('date')
            isDateComplete = not ('?' in event_date) # Ex unfinished date: '????-??-22'
            #does baptism event have a location? ##############################################################################
            event_location = baptism_event.get('location')
            hasLocation = (entry_places is not None) and (len(entry_places)>0)
            
        #is there an identified cleric? ##############################################################################
        hasCleric = not (type(baptism_event.get('cleric'))=='NoneType')
        if hasCleric:
            cleric_PID = baptism_event.get('cleric')
            
        #is there an identified principal? ##############################################################################
        hasPrincipal = not (type(baptism_event.get('principal'))=='NoneType')
            
        #if so:
        if hasPrincipal:
            principal_PID = baptism_event.get('principal')            
            
            #Loop through to find the principal's entry in entry_people
            baptism_princ = [] #Initialization
            for princ_idx in range(len(entry_people)):
                foundPrinc = principal_PID in entry_people[princ_idx].get('id')
                #Need to do "in" because, as per the first case, the listed PID is actually 2 PIDs appended together (separated by a ;)
                if foundPrinc:
                    baptism_princ.append(entry_people[princ_idx])                    
            
            #is the principal an infant? ##############################################################################
            for ref in baptism_princ:
                princ_age = ref.get('age')
                if princ_age == 'infant': 
                    isInfant = 1
                    break
            
            #are principal's godparent(s) identified? ##################################################################
            count_godparents = 0
            godparents_list = []
            count_parents = 0
            parents_list = []
            relations = []
            for ref in baptism_princ:
                if ref.get("relationships") is not None:
                    for rel in ref.get("relationships"):
                        relations.append(rel)            
            if len(relations) > 0:
                hasRelations = 1
                for relation in relations:
                    if relation.get('relationship_type')=='godparent':
                        hasGodparents = 1 
                        godparents_list.append(relation.get('related_person'))
                        count_godparents += 1
                    if (relation.get('relationship_type')=='parent') and (isInfant):
                        hasParents = 1 
                        parents_list.append(relation.get('related_person'))
                        count_parents += 1
                    if relation.get('relationship_type')=='enslaver':
                        hasEnslaver = 1
                        #print("Enslaver found.  This should lead to a true for isEnslaved")
                        #print(relations)
                        #print("------------------")

            #are there likely couples (e.g. parents, godparents) not explicitly flagged as such
            if count_godparents>1:
                for godparent in godparents_list:
                    relations = [person.get('relationships') for person in entry_people if person.get('id')==godparent]
                    if relations is not None: 
                        for rel_idx in range(len(relations)):
                            if relations[0][rel_idx].get('relationship_type')=='husband' or relations[0][rel_idx].get('relationship_type')=='wife':
                                #They are coupled to someone
                                pass
                            else:
                                hasUncoupledGodparents = 1
                del relations
                
            if count_parents>1:
                for parent in parents_list:
                    relations = [person.get('relationships') for person in entry_people if person.get('id')==parent]
                    if relations is not None: 
                        for rel_idx in range(len(relations)):
                            if relations[0][rel_idx].get('relationship_type')=='husband' or relations[0][rel_idx].get('relationship_type')=='wife':
                                continue
                            else:
                                hasUncoupledParents = 1
                                print("Contains uncoupled parents:")
                                print(relations)
                                print("-------------------------")
                del relations
            
        status = ["propiedad", "escrava", "escravos", "esclabo", "esclaba", "escl.a", "escl.o", "clavo", "clava", "escl", "escl.", "escl.s", "clabo", "claba", "esc.va", "esc.ba", "esc.vo", "escvo", "esclavo", "esclava", "escva", "esc.bo", "esclabos", "esclavos", "esc.os", "esc.a", "esc.o", "libre", "esc.s", "esco", "esca"]
        for ref in baptism_princ:
            if ref.get('status') is not None:
                #print("Printing principal status:")
                #print(baptism_princ.get('status'))
                #print("---------------------")
                for term in status:
                    if term in ref.get('status'):
                        isEnslaved = 1
                        break
            if isEnslaved:
                break
        
        ##########################################
        if hasEnslaver and not isEnslaved:
            print("Enslaver found, but thinks principal is not enslaved... Principal dict:")
            print(baptism_princ)
            print("-------------------------")
        
        #IF PRINCIPAL IS AN INFANT:###########################################################################
        if isInfant:
            #are principal's parent(s) identified? #Took care of this above
            #is there a birth event? ##############################################################################
            birth_event = {} #Initialization
            if len(entry_events)>1:
                for birth_idx in range(len(entry_events)):
                    isBirthEvent = entry_events[birth_idx].get('type')=='birth'
                    if isBirthEvent:
                        birth_event = entry_events[birth_idx]
                        #e.g. this assumes there is only one birth event per entry (max)
                        break
            #if so:
            if isBirthEvent:
                #does birth event have a complete date? ##################################################################
                event_date_birth = birth_event.get('date')
                isDateComplete_birth = not ( ('?' in event_date_birth) ) # Unfinished date: '????-??-22'
                #does birth event have a location? #######################################################################
                event_location_birth = birth_event.get('location')
                if (birth_event.get('location') is not None) and (entry_places is not None) and (len(entry_places)>1):
                    hasLocation_birth = 1
                elif (birth_event.get('location') is not None) or (entry_places is not None) or (len(entry_places)>1):
                    hasLocation_birth = 1
                    print("Double check birth location")
                    print(entry_events)
                    print(entry_places)
                    print("--------------------")
                                
        name_list = []        
        #if so, and if principal is enslaved: ##########################################################################
        #is principal's enslaver identified? ########################################################################        
        for person_idx in range(len(entry_people)):
                relationships = entry_people[person_idx].get('relationships')
                if relationships is not None:
                    for relation in relationships:
                        #Check if enslaver has been assigned any ethnicities
                        if (relation.get('relationship_type')=='slave') and (entry_people[person_idx].get('ethnicities') is not None) and (isEnslaved):                            
                            hasWrongEthAssgnt_ensl = 1
                            ################################################
                            ################################################
                            print("Enslaver with assigned ethnicities?")
                            print(entry_people[person_idx])
                            print("-------------------------")
                            ################################################
                            ################################################
                            break
                #Check if cleric has been assigned any ethnicities
                elif (relationships is None) and (entry_people[person_idx].get('id') is cleric_PID) and (entry_people[person_idx].get('ethnicities') is not None):
                    hasWrongEthAssgnt_cler = 1
                    ################################################
                    ################################################
                    print("Cleric with assigned ethnicities?")
                    print(entry_people[person_idx])
                    print("-------------------------")
                    ################################################
                    ################################################
                #Check for people without any role in the event (i.e. no relations, and not the cleric)
                elif (relationships is None) and (entry_people[person_idx].get('id') is not cleric_PID):
                    hasUnrelatedPersons = 1
                    
                #are there any people with very similar names that still appear separately
                nameTemp = entry_people[person_idx].get('name')
                name_list.append(nameTemp)
                #1: Short name that could be a first name
                #Make a boolean list to flag which names may be first names (based on length)
                possibleFirstNames = []
                fullNames = []
                for name in name_list:
                    if name in all_first_names:
                        possibleFirstNames.append(name)
                    else:
                        fullNames.append(name)
                #Check to see if names appear within each other (i.e. is a person double counted)
                doubleCountedNames = []
                for idx in range(len(possibleFirstNames)):
                    doubleCountedNames= doubleCountedNames + ([name for name in fullNames if possibleFirstNames[idx] in name])
                if not len(doubleCountedNames)==0:
                    #print("Possible double count (first name appears in a second instance (full name))")
                    #print(name_list)
                    similarNames = 1
                #2: Two similarly-sized names, that could be variations (i.e. missing hypens or have #'s for unknown letters')
                for name_idx in range(len(name_list)):
                    if name_idx==0:
                        pass
                    else:
                        for idx in range(len(name_list)):
                            for idx2 in range(len(name_list)-idx-1):
                                idx2 = idx2+idx+1
                                if check_lengths(name_list, idx, idx2):
                                    #print(name_list)
                                    similarNames = 1       
                                    
        #questions re characteristics:
        #are there characteristics that were not categorized
        num_rowsC, num_colsC = uncategorized_characteristics.shape
        if num_rowsC>0:
            hasUncatChars = 1
        
        #are there characteristics or relationships that were not assigned
        unassigned_df = check_unassigned(entry_entities)
        num_rowsA, num_colsA = unassigned_df.shape
        #display(unassigned_df)
        if num_rowsA>0:
            hasUnassgnEnts = 1
    else:
        print("Entry is not a baptism, and is not yet supported")
        return {}
    
    my_values = [isBaptism, isDateComplete, hasLocation, hasCleric, hasPrincipal,
              hasEnslaver, isEnslaved, hasRelations, hasGodparents, 
              isInfant, hasParents, isBirthEvent, isDateComplete_birth, hasLocation_birth, 
              hasUnrelatedPersons, hasUncoupledParents, hasUncoupledGodparents, similarNames, 
              hasWrongEthAssgnt_ensl, hasWrongEthAssgnt_cler, hasUncatChars, hasUnassgnEnts]
    
    val_bools = [True if elem==True else False for elem in my_values]
    
    validation_dict = dict(zip(my_keys,val_bools))
    
    #if isVerbose:
        #print(validation_dict)
    
    return validation_dict
    

In [6]:
#export

def process_volume(path_to_transcription, path_to_model):
    '''
    runs the transcription of a single volume (formatted according to SSDA markup 2.0 specs) through the ML entity extraction
    and rules-based relationship linking pipelines, then formats resulting data for export into SQL
        path_to_transcription: path to an XML file containing the transcription of a single volume
        path_to_model: path to a spaCy model trained to extract entities from the proper type of volume
    
        returns: final people, place, and event dictionaries as well as the
        path to a JSON file containing volume metadata as well as people, place, and event records
    '''
    
    #retrieve volume metadata and controlled vocabularies
    
    volume_metadata = retrieve_volume_metadata(path_to_transcription)
    images = xml_v2_to_json(path_to_transcription)
    vocabularies = retrieve_controlled_vocabularies()
    
    if volume_metadata["country"] == "Brazil":
        lang = "pt"
        language = "portuguese"
    else:
        lang = "es"
        language = "spanish"
        
    #load and apply trained model
    
    trained_model = load_model(path_to_model, language=lang, verbose='True')
    
    entry_df = parse_xml_v2(path_to_transcription)
    
    ent_preds_df, metrics_df, per_ent_metrics = test_model(trained_model, entry_df, "entry_no", "text", score_model=False)
    print("Entities extracted.")
    
    #development
    #pd.set_option("display.max_rows", 101)
    #display(ent_preds_df.head(100))
    
    #iterate through each entry and build relationships
    
    people = []
    places = []
    events = []
    
    entitiesRunning = pd.DataFrame()
    noCategoryRunning = pd.DataFrame()
    
    validation_dict_ALL = []
    
    #file path could be passed as parameter, as could language (eventually)
    with open("names.json", encoding="utf-8") as infile:
        name_file = json.load(infile)
        
    names = name_file["names"]
    all_first_names = []
    for name in names:
        all_first_names.append(name["name"])        
    
    for i in range(len(entry_df.index)):
        
        entry_no = entry_df['entry_no'][i]
        entry_text = entry_df['text'][i]    
    
        entities = copy.deepcopy(ent_preds_df[ent_preds_df['entry_no'] == entry_no])
        
        entities["assigned"] = True
        
        entry_people, entry_places, entry_events, entities, characteristics_df, categorized_characteristics, uncategorized_characteristics = build_entry_metadata(entry_text, entities, path_to_transcription, entry_no)             
        
        if uncategorized_characteristics.shape[0] > 0:
            noCategoryRunning = noCategoryRunning.append(uncategorized_characteristics)
        
        #FIND ENTITIES THAT ARE UNASSIGNED OR UNCATEGORIZED
        entity_index = 0
        for ent_data in entities.itertuples():
            for char_data in characteristics_df.itertuples():
                char_index = 0
                #characteristic is not categorized:
                if (char_data.category == None) and (ent_data.pred_start == char_data.pred_start) and (ent_data.pred_entity == char_data.pred_entity):
                    continue #Already dealth with
                #characteristic is categorized but not assigned
                elif (ent_data.pred_label == char_data.pred_label) and (ent_data.pred_start == char_data.pred_start) and (ent_data.pred_entity == char_data.pred_entity):
                    if (char_data.assignment == None):
                        entities.at[entity_index, "assigned"] = False
                char_index += 1
            entity_index += 1

        entitiesRunning = entitiesRunning.append(entities)  
        
        verbosity = 0
        
        entry_validation_dict = validate_entry(entities, entry_people, entry_places, entry_events, uncategorized_characteristics, all_first_names, isVerbose = verbosity)
        validation_dict_ALL.append(entry_validation_dict)
        
        people += entry_people
        places += entry_places
        events += entry_events
    
    noCategoryRunning.reset_index(drop = True, inplace = True)
    noCategoryRunning["assigned"] = False
    print("Relationships linked.")
    
    #disambiguate locations and assign unique ids
    
    unique_places = []
    for place in places:
        if (place != None) and (place not in unique_places):
            unique_places.append(place)
            
    for person in people:        
        if (person["origin"] != None) and (person["origin"] not in unique_places):
            unique_places.append(person["origin"])
    
    places = []
    curr_place = 1
    for unique_place in unique_places:
        place_record = {"id":volume_metadata["id"] + '-L' + str(curr_place), "location":unique_place}
        places.append(place_record)
        curr_place += 1
        
    #incorporate location ids into event metadata and person records
    
    for event in events:
        location = event["location"]
        loc_id = "unknown"
        if location != None:
            for place in places:
                if place["location"] == location:
                    loc_id = place["id"]
        if (loc_id == "unknown") and (location != None):
            print("Failed to find location ID for " + location)
            event["location"] = None
        else:
            event["location"] = loc_id
            
        if event["location"] == "unknown":
            event["location"] = None
            
    for person in people:
        if person["origin"] == None:
            continue
        
        for place in places:
            if place["location"] == person["origin"]:
                person["origin"] = place["id"]
                break
        
    #bracket missing or incomplete event dates
    
    incomplete_dates = []
    last_year = None
    last_month = None
    last_day = None
    
    for e in range(len(events)):
        curr_year = events[e]["date"][:4]
        curr_month = events[e]["date"][5:7]
        curr_day = events[e]["date"][8:]
        
        #fix incompletely extracted years
        if (curr_year != "????") and (last_year != None) and (abs(int(curr_year) - int(last_year)) > 1):
            if (curr_year[3] == last_year[3]):
                curr_year = last_year                
            elif (curr_month == "01") and (last_month == "12"):
                curr_year = str(int(last_year) + 1)                
            else:
                curr_year = last_year
            events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day
        
        if (curr_year == "????") or (curr_month == "??") or (curr_day == "??"):
            #logic to assign dates for birth events based on associated baptism
            if events[e]["type"] == "birth":
                if (events[e]["id"][:events[e]["id"].find('E')] == events[e - 1]["id"][:events[e - 1]["id"].find('E')]) and (events[e - 1]["type"] == "baptism") and ('?' not in events[e - 1]["date"]):
                        if (curr_month != "??") and (curr_day != "??"):
                            if (curr_month == "12") and (last_month == "01"):
                                curr_year = str(int(last_year) - 1)                                
                            elif (30 * int(last_month) + int(last_day) - 30 * int(curr_month) - int(curr_day)) < 21:
                                curr_year = last_year
                            events[e]["date"] = curr_year + '-' + events[e]["date"][5:7] + '-' + events[e]["date"][8:]
                        elif curr_month != "??":
                            if (curr_month == "12"):
                                curr_day = "01"
                                curr_year = str(int(last_year) - 1)
                                events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day + '/' + last_year + '-01-01'
                            elif (curr_month == last_month):
                                curr_day = "01"
                                curr_year = last_year
                                events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day + '/' + last_year + '-' + last_month + '-' + last_day
                            elif int(curr_month) == (int(last_month) - 1):
                                curr_day = "01"
                                curr_year = last_year
                                events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day + '/' + last_year + '-' + last_month + '-01'                            
                        elif curr_day != "??":
                            if curr_day <= last_day:
                                curr_year = last_year
                                curr_month = last_month                                
                            else:
                                if last_month == "01":
                                    curr_month = "12"
                                    curr_year = str(int(last_year) - 1)
                                else:
                                    curr_month = str(int(last_month) - 1)                                    
                                    if len(curr_month) < 2:
                                        curr_month = '0' + curr_month
                                    curr_year = last_year
                            events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day
                        else:
                            if (last_month == '01') and (int(last_day) < 21):
                                curr_year = str(int(last_year) - 1)
                                curr_month = "12"
                                curr_day = str(int(last_day) + 9)                               
                            elif int(last_day) < 21:
                                curr_year = last_year
                                curr_month = str(int(last_month) - 1)
                                if len(curr_month) < 2:
                                    curr_month = '0' + curr_month
                                curr_day = str(int(last_day) + 9)
                            else:
                                curr_year = last_year
                                curr_month = last_month
                                curr_day = str(int(last_day) - 20)
                                if len(curr_day) < 2:
                                    curr_day = '0' + curr_day
                            events[e]["date"] = curr_year + '-' + curr_month + '-' + curr_day + '/' + last_year + '-' + last_month + '-' + last_day
                            
            if (curr_year == "????") or (curr_month == "??") or (curr_day == "??"):
                incomplete_dates.append(e)
        elif last_year == None:
            for date in incomplete_dates:
                events[date]["date"] = complete_date(events[date]["date"], None, curr_year + '-' + curr_month + '-' + curr_day)
            
            incomplete_dates = []
            last_year = curr_year
            last_month = curr_month
            last_day = curr_day
        elif (compare_dates(int(curr_year), int(curr_month), int(curr_day), int(last_year), int(last_month), int(last_day)) == '>') or (compare_dates(int(curr_year), int(curr_month), int(curr_day), int(last_year), int(last_month), int(last_day)) == '='):
            for date in incomplete_dates:
                events[date]["date"] = complete_date(events[date]["date"], last_year + '-' + last_month + '-' + last_day, curr_year + '-' + curr_month + '-' + curr_day)
            
            incomplete_dates = []
            last_year = curr_year
            last_month = curr_month
            last_day = curr_day                    
    
    if last_year != None:
        for date in incomplete_dates:
            events[date]["date"] = complete_date(events[date]["date"], last_year + '-' + last_month + '-' + last_day, None)
        
    #merging any date brackets with equal endpoints
    for event in events:
        interval = event["date"].split('/')
        if (len(interval) == 2) and (interval[0] == interval[1]):
            event["date"] == interval[0]            
        
    print("Events configured.")    
    
    for person in people:        
        #strip titles and/or ranks from names
        if person["name"] != None:
            name_parts = person["name"].split(' ')

            if len(name_parts) >= 2:
                while ((name_parts[0].lower() + ' ' + name_parts[1].lower()) in vocabularies["titles"]) or ((name_parts[0].lower() + ' ' + name_parts[1].lower()) in vocabularies["ranks"]):
                    if len(name_parts) == 2:
                        person["name"] = None
                    else:
                        person["name"] = name_parts[2]
                        for i in range(3, len(name_parts)):
                            person["name"] += ' ' + name_parts[i]

                    if (name_parts[0].lower() + ' ' + name_parts[1].lower()) in vocabularies["titles"]:
                        if person["titles"] != None:
                            person["titles"] += ';' + name_parts[0] + ' ' + name_parts[1]
                        else:
                            person["titles"] = name_parts[0] + ' ' + name_parts[1]
                    else:
                        if person["ranks"] != None:
                            person["ranks"] += ';' + name_parts[0] + ' ' + name_parts[1]
                        else:
                            person["ranks"] = name_parts[0] + ' ' + name_parts[1]

                    if person["name"] == None:
                        break
                    name_parts = person["name"].split(' ')
                    if len(name_parts) < 2:
                        break

            if person["name"] != None:
                while (name_parts[0].lower() in vocabularies["titles"]) or (name_parts[0].lower() in vocabularies["ranks"]):
                    if len(name_parts) == 1:
                        person["name"] = None
                    else:
                        person["name"] = name_parts[1]
                        for i in range(2, len(name_parts)):
                            person["name"] += ' ' + name_parts[i]

                    if name_parts[0].lower() in vocabularies["titles"]:
                        if person["titles"] != None:
                            person["titles"] += ';' + name_parts[0]
                        else:
                            person["titles"] = name_parts[0]
                    else:
                        if person["ranks"] != None:
                            person["ranks"] += ';' + name_parts[0]
                        else:
                            person["ranks"] = name_parts[0]

                    if person["name"] == None:
                        break
                    name_parts = person["name"].split(' ')
                    
    #normalize names and all characteristics
    names = []
    name_counts = []
    ethnonym_vocab = retrieve_json_vocab("synonyms.json", "ethnonyms")
    phenotype_vocab = retrieve_json_vocab("synonyms.json", "phenotypes", language="spanish")
    
    for person in people:
        #normalize characteristics and translate to English
        for key in person:
            if person[key] == None:
                continue
            if key == "name":
                person[key] = normalize_text(person[key], "synonyms.json", context="name")
                #check extracted name for ethnonyms and/or attributed phenotypes        
                if (person["name"] != None) and (person["name"] != normalize_text(person["name"], "synonyms.json", context="ethnonym")):
                    for token in person["name"].split(' '):
                        eth_norm = normalize_text(token, "synonyms.json", context="ethnonym")
                        if token != eth_norm:
                            if (person["ethnicities"] == None) or (not (eth_norm in person["ethnicities"])):
                                if person["ethnicities"] == None:
                                    person["ethnicities"] = eth_norm
                                else:
                                    person["ethnicities"] = person["ethnicities"] + ';' + eth_norm
                    person["name"] = normalize_text(person["name"], "synonyms.json", context="ethnonym")
                else:
                    for ethnonym in ethnonym_vocab:
                        if ethnonym in person["name"]:
                            if person["ethnicities"] == None:
                                person["ethnicities"] = ethnonym
                            else:
                                person["ethnicities"] = person["ethnicities"] + ';' + ethnonym
                for phenotype in phenotype_vocab:
                    if phenotype in normalize_text(person[key], "synonyms.json", context="characteristic"):                    
                        if person["phenotype"] == None:
                            person["phenotype"] = phenotype
                        else:
                            person["phenotype"] = person["phenotype"] + ';' + phenotype
                        if phenotype[-1] == 's':
                            for token in person["name"].split(' '):
                                if normalize_text(token, "synonyms.json", context="characteristic") == phenotype:
                                    person["name"] = person["name"].replace(' ' + token, '')
            elif key == "ethnicities":                
                if person[key].find(';') == -1:
                    person[key] = normalize_text(person[key], "synonyms.json", context="ethnonym")                    
                else:
                    char_comp = person[key].split(';')
                    person[key] = ""
                    #strip out duplicate characteristics
                    for char in char_comp:
                        char = normalize_text(char, "synonyms.json", context="ethnonym")                       
                                          
                        if not (char in person[key]):
                            if person[key] == "":
                                person[key] = char
                            else:
                                person[key] = person[key] + ';' + char
            elif (key != "id") and (key != "relationships"):
                if person[key].find(';') == -1:
                    person[key] = normalize_text(person[key], "synonyms.json", context="characteristic")
                    person[key] = translate_characteristic(person[key], "synonyms.json", language)
                else:
                    char_comp = person[key].split(';')
                    person[key] = ""
                    #strip out duplicate characteristics
                    for char in char_comp:
                        char = normalize_text(char, "synonyms.json", context="characteristic")                        
                        char = translate_characteristic(char, "synonyms.json", language)                        
                        if not (char in person[key]):
                            if person[key] == "":
                                person[key] = char
                            else:
                                person[key] = person[key] + ';' + char           
        
        #future improvement: find additional references for plural characteristics
        
        #count name frequency
        if person["name"] != None:
            if person["name"] in names:
                name_counts[names.index(person['name'])] += 1
            else:
                names.append(person["name"])
                name_counts.append(1)   
    
    #disambiguate and merge people across the volume
    redundant_records = []
    merged_records = []    
    for i in range(len(name_counts)):
        if (name_counts[i] > .1 * len(images)) and (len(names[i].split(' ')) > 1) and (names[i] != "Unknown principal"):
            records_to_merge = []            
            for j in range(len(people)):
                if people[j]["name"] == names[i]:
                    redundant_records.append(people[j])
                    records_to_merge.append(people[j])                    
            merged_records.append(merge_records(records_to_merge))            
    people = [person for person in people if person not in redundant_records]
    for person in merged_records:
        people.append(person)    
    
    print("People records enhanced and disambiguated.")
    
    #reduce compound person IDs to single ID, add references field
    people, events = compact_references(people, events)
    
    print("Single ID generated for each individual.")
    
    #convert dictionaries into JSON    
    with open("volume_records\\" + volume_metadata["id"] + ".json", "w") as outfile:
        outfile.write('{\n\"volume\": \n')
        json.dump(volume_metadata, outfile)
        outfile.write(',')
        outfile.write('\n\"images\": [\n')
        first_img = True
        for image in images:
            if first_img:
                first_img = False
            else:
                outfile.write(",\n")
            json.dump(image, outfile)
        outfile.write("\n],\n")
        outfile.write('\n\"people\": [\n')
        first_person = True
        for person in people:
            if first_person:
                first_person = False
            else:
                outfile.write(",\n")            
            json.dump(person, outfile)            
        outfile.write("\n],\n")
        outfile.write("\"places\": [\n")
        first_place = True
        for place in places:
            if first_place:
                first_place = False
            else:
                outfile.write(",\n")
            json.dump(place, outfile)
        outfile.write("\n],\n")
        outfile.write("\"events\": [\n")
        first_event = True
        for event in events:
            if first_event:
                first_event = False
            else:
                outfile.write(",\n")
            json.dump(event, outfile)
        outfile.write("\n]\n")
        outfile.write('}')
        
    #dump validation dictionaries
    with open("validation\\" + volume_metadata["id"] + ".json", "w") as outfile:
        outfile.write('{\n\"entries\": [\n')
        first_entry = True
        for entry in validation_dict_ALL:
            if first_entry:
                first_entry = False
            else:
                outfile.write(",\n")
            json.dump(entry, outfile)
        outfile.write("\n]\n")
        outfile.write('}')
            
    print("JSON built, processing completed.")
            
    return people, places, events, volume_metadata["id"] + "_ppe.json", entitiesRunning, noCategoryRunning, validation_dict_ALL

In [7]:
#no_test

people, places, events, json_path, entities, noCategory, validation_list = process_volume("transcriptions\\15834.xml", "models/15834")

C:\Users\kdmen\miniconda3\envs\ssda\lib\site-packages\spacy\util.py:732: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.1.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Loaded model 'models/15834'
Entities extracted.
Enslaver found, but thinks principal is not enslaved... Principal dict:
[{'id': '15834-1034-3-P1;15834-1034-3-P3', 'name': 'Antonia', 'origin': None, 'ethnicities': 'mina', 'age': 'adulta', 'legitimacy': None, 'occupation': None, 'phenotype': 'negra', 'status': 'Esc.va', 'titles': None, 'ranks': None, 'relationships': [{'related_person': '15834-1034-3-P5', 'relationship_type': 'godparent'}, {'related_person': '15834-1034-3-P4', 'relationship_type': 'enslaver'}]}]
-------------------------
Enslaver with assigned ethnicities?
{'id': '15834-1039-4-P4', 'name': 'D. Ju.o Joseph de Justis', 'origin': None, 'ethnicities': 'Mina', 'age': None, 'legitimacy': None, 'occupation': None, 'phenotype': None, 'status': None, 'titles': None, 'ranks': None, 'relationships': [{'related_person': '15834-1039-4-P3', 'relationship_type': 'slave'}]}
-------------------------
Enslaver found, but thinks principal is not enslaved... Principal dict:
[{'id': '15834-1

KeyError: 10

In [ ]:
#no_test

unassigned_df = check_unassigned(entities, isVerbose=1)

In [ ]:
#export

def flatten_volume_json(path_to_volume_json, csv_root=''):
    '''
    flattens JSON record for a volume into six separate CSVs (volume, entries, people, relationships, places, and events)
        path_to_volume_json: path to a volume JSON record
        csv_root: specify directory for CSV output, including trailing /
    
        returns: root directory for CSVs
    '''    
    
    with open(path_to_volume_json, encoding="utf-8") as jsonfile:
        data = json.load(jsonfile)
        
    volume_id = data["volume"]["id"]
    
    with open(csv_root + volume_id + "_volume.csv", 'w', encoding="utf-8") as outfile:
        keys = 0
        for key in data["volume"]:
            outfile.write(key)
            keys += 1
            if keys == len(data["volume"]):
                outfile.write('\n')
            else:
                outfile.write(',')
        keys = 0
        for key in data["volume"]:
            outfile.write('"' + data["volume"][key] + '"')            
            keys += 1
            if keys == len(data["volume"]):
                break
            else:
                outfile.write(',')
                
    with open(csv_root + volume_id + "_entries.csv", 'w', encoding="utf-8") as outfile:
        outfile.write("entry id,entry text\n")
        for image in data["images"]:                        
            image_id = volume_id + '-' + image["id"]            
            for entry in image["entries"]:
                entry_id = image_id + '-' + str(entry["id"])
                entry_text = entry["text"]
                outfile.write(entry_id + ',' + '"' + entry_text + '"\n')
                
    with open(csv_root + volume_id + "_people.csv", 'w', encoding="utf-8") as outfile:
        outfile.write("id,name,origin,ethnicity,age,legitimacy,occupation,phenotype,status,titles,ranks,references\n")
        relationships = []
        for person in data["people"]:
            for key in person:
                if key == "relationships": 
                    if person[key] == None:
                        continue
                    for relationship in person[key]:                       
                        if relationship["relationship_type"] == "godchild":
                            inverse_relationship_type = "godparent"
                        elif relationship["relationship_type"] == "godparent":
                            inverse_relationship_type = "godchild"
                        elif relationship["relationship_type"] == "grandparent":
                            inverse_relationship_type = "grandchild"
                        elif relationship["relationship_type"] == "grandchild":
                            inverse_relationship_type = "grandparent"
                        elif relationship["relationship_type"] == "parent":
                            inverse_relationship_type = "child"
                        elif relationship["relationship_type"] == "child":
                            inverse_relationship_type = "parent"
                        elif relationship["relationship_type"] == "slave":
                            inverse_relationship_type = "enslaver"
                        elif relationship["relationship_type"] == "enslaver":
                            inverse_relationship_type = "slave"
                        else:
                            inverse_relationship_type = relationship["relationship_type"]
                            
                        inverse_relationship = {"from": relationship["related_person"], "to": person["id"], "type": inverse_relationship_type}
                        if not (inverse_relationship in relationships):
                            relationships.append({"from": person["id"], "to": relationship["related_person"], "type": relationship["relationship_type"]})
                        
                elif key == "references":
                    references = person[key][0]
                    for index in range(1, len(person[key])):
                        references += ';' + person[key][index]
                    outfile.write(references + '\n')
                elif person[key] == None:
                    outfile.write(',')
                else:
                    outfile.write(person[key] + ',')
                    
    with open(csv_root + volume_id + "_relationships.csv", 'w', encoding="utf-8") as outfile:
        outfile.write("from id,to id,relationship type\n")
        for relationship in relationships:
            outfile.write(relationship["from"] + ',' + relationship["to"] + ',' + relationship["type"] + '\n')
            
    with open(csv_root + volume_id + "_places.csv", 'w', encoding="utf-8") as outfile:
        outfile.write("id,location\n")
        for place in data["places"]:
            outfile.write(place["id"] + ',' + place["location"] + '\n')
            
    with open(csv_root + volume_id + "_events.csv", 'w', encoding="utf-8") as outfile:
        outfile.write("id,type,principal,date,location id,cleric\n")
        for event in data["events"]:
            for key in event:
                if event[key] == None:
                    event[key] = ''
            outfile.write(event["id"] + ',' + event["type"] + ',' + event["principal"] + ',' + event["date"] + ',' + event["location"] + ',' + event["cleric"] + '\n')   
                
    return csv_root

In [ ]:
#no_test

flatten_volume_json("volume_records/15834.json", csv_root = "volume_records/csv/")

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()